In [1]:
pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163342 sha256=c586ec5b5d3c6bb69b199a0b57f79c0e80aa8816ee5df7a3a409d2774e0d86f3
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
from surprise import Dataset, Reader, KNNBasic
import pandas as pd

# Sample data (user_id, song_id, rating)
data = {'user_id': [1, 1, 2, 2, 3],
        'song_id': ['A', 'B', 'A', 'C', 'B'],
        'rating': [5, 4, 5, 3, 4]}

df = pd.DataFrame(data)

# Define the rating scale
reader = Reader(rating_scale=(1, 5))

# Load the dataset
dataset = Dataset.load_from_df(df[['user_id', 'song_id', 'rating']], reader)

# Build the training set
trainset = dataset.build_full_trainset()

# Use the KNNBasic collaborative filtering algorithm
sim_options = {'name': 'cosine', 'user_based': True}
model = KNNBasic(sim_options=sim_options)

# Train the model
model.fit(trainset)

# Generate a playlist for a given user
def generate_playlist(user_id, n=5):
    # Get all items the user has not rated
    all_songs = df['song_id'].unique()
    user_rated_songs = df[df['user_id'] == user_id]['song_id'].tolist()
    unrated_songs = list(set(all_songs) - set(user_rated_songs))

    # Predict ratings for unrated songs
    predictions = [model.predict(user_id, song_id) for song_id in unrated_songs]

    # Sort predictions by estimated rating
    sorted_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)

    # Get the top N recommendations
    top_n = sorted_predictions[:n]

    # Extract song IDs from recommendations
    recommended_songs = [pred.iid for pred in top_n]

    return recommended_songs

# Example usage
user_id = 1
playlist = generate_playlist(user_id)
print(f"Recommended playlist for user {user_id}: {playlist}")


Computing the cosine similarity matrix...
Done computing similarity matrix.
Recommended playlist for user 1: ['C']
